Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.
Обучите по коллекции предложений word2vec

Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    тест на определение ближайших слов
    тест на аналогии (мужчина – король : женщина – королева)
    тест на определение лишнего слова.

Постройте несколько визуализаций:
    TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.


In [1]:
import json

import bz2
import re
from tqdm import tqdm
from scipy import sparse

In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# from pymystem3 import Mystem - too slow
from pymorphy2 import MorphAnalyzer

from gensim.models import word2vec

In [4]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [04:17, 781.59it/s] 


In [5]:
responses[10]

{'city': 'г. Москва',
 'rating_not_checked': False,
 'title': 'Повесили кредит',
 'num_comments': 15,
 'bank_license': 'лицензия № 1326',
 'author': 'Юля2015',
 'bank_name': 'Альфа-Банк',
 'datetime': '2015-06-05 12:24:28',
 'text': 'Вот даже и засомневалась, какую ставить оценку... в итоге пишу без неё. Являюсь клиентом Альфа-Банка с 2012 года. Взяла тогда кредит наличными. Погашаю его, в т.ч. частично-досрочными суммами. Потом мне выпустили кредитную карту, на весьма выгодных условиях. Ей я тоже пользуюсь и вношу платежи. И вот на днях звонит мне девушка и говорит, что у меня просрочка. Я, оставаясь с ней на связи, лезу в яндекс.деньги и проверяю: карта и кредит оплачены. И тут она мне выдаёт: "У Вас в нашем банке 3(!) кредитных продукта! В апреле 2014 Вы брали потребительский кредит на продукт". Я в шоке, т.к. никаких кредитов в Альфе я больше не брала. Мне говорят, что нужно прийти в отделение и написать претензию. Кладу трубку. Вспоминаю: в апреле прошлого года я подавала заявку н

In [6]:
ratings = set()
for resp in tqdm(responses):
    ratings.add(resp['rating_grade'])

ratings

100%|██████████████████████████████| 153499/153499 [00:00<00:00, 629058.72it/s]


{1, 2, 3, 4, 5, None}

In [7]:
ratings

{1, 2, 3, 4, 5, None}

## Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. Обучите по коллекции предложений word2vec

In [8]:
my_stop = stopwords.words('russian')
ru_words = re.compile("[А-Яа-яёЁ]+")
#Will use pymorphy - mystem is too slow on Windows, takes more than 1 minute to process 1 comment
pymorphy2_analyzer = MorphAnalyzer()

def words_only(text):
    return " ".join(ru_words.findall(text))

def lemmatize(text, lemmatizer = pymorphy2_analyzer):
    out_tok = []
    try:
        tok_list = word_tokenize(text)
        for w in tok_list:
            out_tok.append(lemmatizer.parse(w)[0].normal_form)
        return " ".join(out_tok)  
    except:
        return " "  


def remove_stopwords(text, mystopwords = my_stop):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""
    
def preprocess(text):
    output = []
    sentetnces = sent_tokenize(text.lower())
    
    for cur in sentetnces:
        output.append(remove_stopwords(lemmatize(words_only(cur))))
        
    return output

In [ ]:
sent_list = []
for text in tqdm(responses):
    sent_list.append(preprocess(text['text']))
    
#flattern list 
flat_sent_list = [item for sublist in sent_list for item in sublist]

In [ ]:
#Saving to file
with open('sentences.txt', mode='w') as f:
    for cur in flat_sent_list:
        f.write(cur + '\n')

In [9]:
#Loading from file
flat_sent_list = []
with open('sentences.txt', mode='r') as f:
    for line in f:
        flat_sent_list.append(line)

In [10]:
sent_list_tokens = []
for cur in flat_sent_list:
    sent_list_tokens.append(cur[:-1].split())
    
# sent_list_tokens[5]

In [11]:
#Training model
%time model_ru = word2vec.Word2Vec(sent_list_tokens, workers=4, size=300, min_count=10, window=10, sample=1e-3)

Wall time: 6min 49s


In [12]:
len(model_ru.wv.vocab)

24106

### Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec: тест на определение ближайших слов тест на аналогии (мужчина – король : женщина – королева) тест на определение лишнего слова.

In [13]:
# Видно, что антоним очень близко
model_ru.wv.most_similar(positive=['хороший'])

[('плохой', 0.6474036574363708),
 ('неплохой', 0.6050633192062378),
 ('отличный', 0.5753710269927979),
 ('выгодный', 0.5591389536857605),
 ('привлекательный', 0.5584959387779236),
 ('хорошеть', 0.5550081729888916),
 ('достойный', 0.525431215763092),
 ('классный', 0.5222017765045166),
 ('надёжный', 0.4992052912712097),
 ('комфортный', 0.4900427758693695)]

In [14]:
model_ru.wv.most_similar(positive=['плохой', 'альфа'], negative=['хороший'])

[('отп', 0.5756067633628845),
 ('скб', 0.4401751756668091),
 ('мдм', 0.41588273644447327),
 ('номос', 0.4117511510848999),
 ('восточный', 0.39711815118789673),
 ('балтийский', 0.3919181525707245),
 ('итб', 0.3917805552482605),
 ('сетель', 0.38498425483703613),
 ('росгосстрах', 0.380077064037323),
 ('ренессанс', 0.3697563111782074)]

In [15]:
model_ru.wv.most_similar(positive=['женщина', 'сотрудник'], negative=['мужчина'])

[('работник', 0.7385704517364502),
 ('сотрудница', 0.7382783889770508),
 ('специалист', 0.6084808111190796),
 ('менеджер', 0.5310217142105103),
 ('персонал', 0.5257631540298462),
 ('работница', 0.5001120567321777),
 ('девушка', 0.48925039172172546),
 ('девочка', 0.4627029597759247),
 ('барышня', 0.45460212230682373),
 ('оператор', 0.4542313814163208)]

In [16]:
# Неправильные аналогии - не думаю, что слова король и королева часто встречаются в тескте
model_ru.wv.most_similar(positive=['женщина', 'король'], negative=['мужчина'])

[('дурачок', 0.5484744906425476),
 ('детски', 0.5454837083816528),
 ('прослезиться', 0.5221619606018066),
 ('соответствующе', 0.5178133845329285),
 ('опрятно', 0.5157357454299927),
 ('правдоподобно', 0.5109726786613464),
 ('лукавство', 0.5099987983703613),
 ('свита', 0.5086867809295654),
 ('неладно', 0.5085864067077637),
 ('мания', 0.5065679550170898)]

In [17]:
model_ru.wv.doesnt_match(['сотрудник', 'работник', 'менеджер', 'клиент'])

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'клиент'

### Постройте несколько визуализаций: TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

In [18]:
from sklearn.manifold import TSNE
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

In [19]:
N_WORDS = 500
top_words = model_ru.wv.index2entity[:N_WORDS]
top_words_vec = model_ru.wv.__getitem__(top_words)

In [20]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

Wall time: 10.3 s


In [21]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (rus model, top words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

### Видны четкие кластеры по месяцам года и операциям (снять-внести)

## По осям хорошо-плохо и медленно-быстро

In [22]:
#лемматизатор отработал так себе... слова "хорошо" нет, зато есть слова "плохо" и "плохой"
good_bad = model_ru.wv.__getitem__('хороший') - model_ru.wv.__getitem__('плохой')
fast_slow = model_ru.wv.__getitem__('быстрый') - model_ru.wv.__getitem__('медленный')

In [23]:
gb_axis = np.matmul(top_words_vec, good_bad)
fs_axis = np.matmul(top_words_vec, fast_slow)

In [24]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (rus model, GB & FS axis)")

source = ColumnDataSource(data=dict(x1=gb_axis,
                                    x2=fs_axis,
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

### слова выстроились по осям